## Reproject Datasets

Here is a link for a reprojectiong method from a HyTEST notebook: [conus404_regrid.ipynb](https://github.com/hytest-org/hytest/blob/main/dataset_access/conus404_regrid.ipynb)

Uses xESMF to reproject rectilinear grids, which is what we have. See this [notebook](https://xesmf.readthedocs.io/en/latest/notebooks/Rectilinear_grid.html) for details. Uses one of six algorithms listed [here](https://xesmf.readthedocs.io/en/latest/notebooks/Compare_algorithms.html). They recommend `conservative` with upscaling (increased pixel size). An inportant note is that extra dimensions must be on the left, i.e. `(time, lev, lat, lon)` is correct but `(lat, lon, time, lev)` would not work. Most data sets should have `(lat, lon)` on the right (being the fastest changing dimension in the memory). If not, use `DataArray.transpose` or `numpy.transpose` to preprocess the data.

xESMF can also extrapolate holes in the data set, which may be needed. See this [notebook](https://xesmf.readthedocs.io/en/latest/notebooks/Masking.html#Extrapolation).

In [ ]:
import hvplot.xarray
import cartopy.crs as ccrs
import numpy as np
import xarray as xr
import os
from pathlib import Path

if 'ESMFMKFILE' not in os.environ:
    os.environ['ESMFMKFILE'] = str(Path(os.__file__).parent.parent / 'esmf.mk')

import xesmf as xe

In [ ]:
ds = xr.tutorial.open_dataset(
    "air_temperature"
)  # use xr.tutorial.load_dataset() for xarray<v0.11.0
ds

In [ ]:
dr = ds["air"]  # get a DataArray
# https://hvplot.holoviz.org/user_guide/Geographic_Data.html
dr.isel(time=0).hvplot(geo=True, coastline=True)

In [ ]:
ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(15, 75, 5), {"units": "degrees_north"}),
        "lon": (["lon"], np.arange(200, 330, 5), {"units": "degrees_east"}),
    }
)
ds_out

In [ ]:
regridder = xe.Regridder(ds, ds_out, "conservative")
regridder  # print basic regridder information.

In [ ]:
dr_out = regridder(dr, keep_attrs=True)
dr_out

In [ ]:
dr_out.isel(time=0).hvplot(geo=True, coastline=True, width=500) + dr.isel(time=0).hvplot(geo=True, coastline=True, width=500)

In [ ]:
# Double check time series is the same
dr.sel(lon=260, lat=40).hvplot() * dr_out.sel(lon=260, lat=40).hvplot()